In [1115]:
import numpy as np

In [1116]:
class ANN:
    w_l_ij= None
    b_l_j = None
    a_l_ij = None
    delta_l_ij = None
    input_featureCount = None
    input_count = None
    layer_info = None
    dl_dw = None
    def __init__(self,xShape,rseed = 10,listLayer= None):
        np.random.seed(rseed)
        inputCount,inputDim = xShape
        self.input_featureCount = inputDim
        self.input_count = inputCount
        self.layer_info = [['None',inputDim]]
        self.w_l_ij = ['None']
        self.b_l_j = ['None']
        self.optimizer = None

    def compile(self,optimizer,lossFunction):
        self.optimizer = optimizer
        self.layer_info.append(['Loss',lossFunction])

    def __str__(self):
        return

    def addLayers_Dense(self,neuronCount,activationFunction):
        if(len(self.layer_info) == 1) :
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.input_featureCount,neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)
        else:
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.layer_info[-2][2],neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)

    def forward_layer_dense(self,layerNo):
        if layerNo < 1:
            print("ERRRROR")
        output = self.layer_info[layerNo][1](self.a_l_ij[layerNo-1] @ self.w_l_ij[layerNo] + self.b_l_j[layerNo]) 
        self.a_l_ij.append(output)

    def forward_layer_loss(self,layerNo,y):
        y_hat  =  self.a_l_ij[-1]
        lossFunction = self.layer_info[layerNo][1]
        return lossFunction(y_hat,y)

    def forwardProp(self,x,y):
        self.a_l_ij = []
        self.a_l_ij.append(x)
        for layerNo in range(len(self.layer_info)):
            if self.layer_info[layerNo][0] == 'Dense':
                self.forward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Loss':
                cost = self.forward_layer_loss(layerNo,y)
                return cost
            
    def backwardProp(self,y):
        self.dl_dw = []
        lastIndex = len(self.layer_info) - 2
        self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        self.delta_l_ij[lastIndex] = (2 * (self.a_l_ij[lastIndex] - y).T * self.diff_activation_linear(self.a_l_ij[lastIndex]))
        for layerNo in range(lastIndex,0,-1):
            self.dl_dw.append(self.a_l_ij[layerNo-1].T @ self.delta_l_ij[layerNo])
            self.delta_l_ij[layerNo-1] = (self.w_l_ij[layerNo] @self.delta_l_ij[layerNo].T).T * self.diff_activation_linear(self.a_l_ij[layerNo-1])
        self.dl_dw.append(['None'])
        self.dl_dw.reverse()

    def fit(self,x,y):
        #Hyperparameters if required: 
        epoch = 21
        eta = 1e-4
        self.optimizer(self,x,y,eta,epoch)

    @staticmethod    
    def optimizer_gradientDescent(Obj,x,y,eta,epoch):
        for i in  range(epoch):
            errorSum = 0
            print("Epoch:",i,end="\t")
            for j in range(y.shape[1]):
                #x and y both are 2d matrix
                x_1xd = x[j:j+1]
                y_1xk = y[j:j+1]
                errorSum += Obj.forwardProp(x_1xd,y_1xk)
                Obj.backwardProp(y_1xk)
                for index in range(1,len(Obj.w_l_ij)):
                    Obj.w_l_ij[index] = Obj.w_l_ij[index] - (eta * Obj.dl_dw[index])
                    Obj.b_l_j[index] = Obj.b_l_j[index] - (eta * Obj.delta_l_ij[index])
            print("Error:",errorSum)

    @staticmethod
    def loss_MSE(P,y):
        return np.sum((P - y)**2)   
      
    @staticmethod
    def activation_linear(z):
        return z

    def diff_activation_linear(self,x):
        return np.ones(x.shape)
    
    @staticmethod
    def activation_sigmoid(z):    
        return 1 / (1 + np.exp(-z))
    
    @staticmethod
    def activation_tanh(z):
        pass

    @staticmethod
    def activation_relu(z):
        pass

    @staticmethod
    def activation_softmax(z):
        pass



    


In [1117]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [1118]:
# dictt = sk.fetch_california_housing()
# x = dictt.data
# y = np.array([dictt.target])
# y = y.T
# x[0:1].shape

In [1119]:
x = np.array([[1,2,3,4,5],[6,7,8,9,10]])
y = np.array([[1],[2]])

In [1120]:
model = ANN(x.shape,20)

In [1121]:
model.addLayers_Dense(3,ANN.activation_linear)
model.addLayers_Dense(4,ANN.activation_linear)
model.addLayers_Dense(1,ANN.activation_linear)

In [1122]:
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)

In [1123]:
model.layer_info

[['None', 5],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 3],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 4],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 1],
 ['Loss', <function __main__.ANN.loss_MSE(P, y)>]]

In [1124]:
model.delta_l_ij

In [1125]:
model.b_l_j

['None',
 array([[0.03666431, 0.11669374, 0.7512807 ]]),
 array([[0.67941112, 0.65078591, 0.26879524, 0.06732467]]),
 array([[0.26362883]])]

In [1126]:
model.dl_dw

In [1127]:
model.a_l_ij

In [1128]:
import copy
w_old = copy.deepcopy(model.w_l_ij)

In [1129]:
model.fit(x,y)

Epoch: 0	Error: 1044.8028100300546
Epoch: 1	Error: 514.711385006634
Epoch: 2	Error: 306.3701947013887
Epoch: 3	Error: 198.3185123751238
Epoch: 4	Error: 134.56775022710355
Epoch: 5	Error: 94.0373777500907
Epoch: 6	Error: 67.01053681329807
Epoch: 7	Error: 48.39892446403348
Epoch: 8	Error: 35.29134955181806
Epoch: 9	Error: 25.9110220086745
Epoch: 10	Error: 19.1196594922008
Epoch: 11	Error: 14.160667607998343
Epoch: 12	Error: 10.516787728185248
Epoch: 13	Error: 7.826678387690066
Epoch: 14	Error: 5.83371694187931
Epoch: 15	Error: 4.353338256947902
Epoch: 16	Error: 3.2515177010257856
Epoch: 17	Error: 2.4302146223598418
Epoch: 18	Error: 1.8173091128599674
Epoch: 19	Error: 1.3595225708357563
Epoch: 20	Error: 1.0173675744844826


In [1130]:
w_new = model.w_l_ij

In [1131]:
w_old,w_new

(['None',
  array([[0.5881308 , 0.89771373, 0.89153073],
         [0.81583748, 0.03588959, 0.69175758],
         [0.37868094, 0.51851095, 0.65795147],
         [0.19385022, 0.2723164 , 0.71860593],
         [0.78300361, 0.85032764, 0.77524489]]),
  array([[0.23921822, 0.25480601, 0.85762553, 0.94977903],
         [0.56168686, 0.17878052, 0.77025193, 0.49238104],
         [0.63125307, 0.83949792, 0.4610394 , 0.49794007]]),
  array([[0.77144514],
         [0.48098413],
         [0.32920641],
         [0.51064106]])],
 ['None',
  array([[0.57218926, 0.88073125, 0.86846012],
         [0.7839544 , 0.00192464, 0.64561636],
         [0.33085633, 0.46756353, 0.58873963],
         [0.13008407, 0.20438651, 0.62632349],
         [0.70329592, 0.76541527, 0.65989183]]),
  array([[0.09593223, 0.18155081, 0.83697849, 0.88036705],
         [0.42025752, 0.10644068, 0.74978754, 0.42381524],
         [0.42142828, 0.73227034, 0.43091972, 0.39636726]]),
  array([[ 0.40842161],
         [ 0.10173592],
     